In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras .layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

In [2]:
df = pd.read_csv('pokemon.csv')
df2 = df.sort_values(by=['Name'], ascending=True).reset_index(drop=True)
df2.head()

,Name,Type1,Type2
0,abomasnow,Grass,Ice
1,abra,Psychic,NaN
2,absol,Dark,NaN
3,accelgor,Bug,NaN
4,aegislash-blade,Steel,Ghost


In [3]:
#image pathlerini data'mıza ekleyelim
import os
path='images/'
image = sorted(os.listdir('images/'))
image_paths = []
for i in image:
    image_paths.append(path + i)
df2['imagepath'] = image_paths

#type 1 ve type 2 için iki ayrı frame oluşturuyoruz
type1_data = df2.drop((['Type2']), axis=1)
type2_data = df2.drop((['Type1']), axis=1)
#type 2 de eksik data olduğu için çıkarıyoruz
type2_data = type2_data.dropna()
#type 2 de çok fazla none data var. az data olduğu için sadece type1 i algoritmaya dahil ettim.
#type1_data['Type1'].value_counts()
type1_data.Type1.value_counts().keys()

Index(['Water', 'Normal', 'Grass', 'Bug', 'Psychic', 'Fire', 'Rock',
       'Electric', 'Poison', 'Ground', 'Dark', 'Fighting', 'Ghost', 'Dragon',
       'Steel', 'Ice', 'Fairy', 'Flying'],
      dtype='object')

In [4]:
type1_data.Type1.value_counts().values

array([114, 105,  78,  72,  53,  53,  46,  40,  34,  32,  29,  29,  27,
        27,  26,  23,  18,   3], dtype=int64)

In [5]:
select = ['Water', 'Normal', 'Grass', 'Bug']
type1_data = type1_data[type1_data['Type1'].isin(select)]
#type 1'i train ve test olarak ayırıyoruz
train_type1_df = type1_data.sample(frac=0.85, random_state = 0)
test_type1_df = type1_data.drop(train_type1_df.index)
train_type1_df.head(n=5)

,Name,Type1,imagepath
334,kangaskhan,Normal,images/kangaskhan.png
351,kricketune,Bug,images/kricketune.png
623,silvally,Normal,images/silvally.jpg
118,combee,Bug,images/combee.png
676,suicune,Water,images/suicune.png


In [6]:
train_datagen = ImageDataGenerator(
    
    rescale=1./255, 
    rotation_range=40, 
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.1
)


train_generator = train_datagen.flow_from_dataframe(
    train_type1_df, 
    x_col='imagepath', y_col='Type1',
    target_size=(120, 120), 
    color_mode='rgba', 
    class_mode='categorical', 
    batch_size=32, 
    shuffle=True, 
    seed=1,
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    train_type1_df , x_col='imagepath', y_col='Type1',
    target_size=(120, 120),
    color_mode='rgba', 
    class_mode='categorical', 
    batch_size=4, 
    shuffle=True,
    seed=1, 
    subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_type1_df,
    x_col="imagepath", y_col=None,
    target_size=(120, 120),
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=1  
)



Found 283 validated image filenames belonging to 4 classes.
Found 31 validated image filenames belonging to 4 classes.
Found 55 validated image filenames.


In [7]:
model = Sequential()

model.add(Dense(4, input_shape=(120, 120, 4)))

model.add(Conv2D(64, (7, 7), activation='relu'))
model.add(MaxPooling2D(2, 2))


model.add(Conv2D(128, (7, 7), activation='relu'))
model.add(MaxPooling2D(2, 2))


model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(4, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120, 120, 4)       20        
                                                                 
 conv2d (Conv2D)             (None, 114, 114, 64)      12608     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 57, 57, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 51, 51, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 128)      0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 80000)             0

In [8]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=80
)


Epoch 1/80
8/8 [==============================] - 14s 2s/step - loss: 1.9773 - acc: 0.2948 - val_loss: 1.3892 - val_acc: 0.1786
Epoch 2/80
8/8 [==============================] - 15s 2s/step - loss: 1.3858 - acc: 0.2869 - val_loss: 1.4112 - val_acc: 0.2857
Epoch 3/80
8/8 [==============================] - 15s 2s/step - loss: 1.3562 - acc: 0.3028 - val_loss: 1.2729 - val_acc: 0.3929
Epoch 4/80
8/8 [==============================] - 15s 2s/step - loss: 1.2868 - acc: 0.3625 - val_loss: 1.1690 - val_acc: 0.5000
Epoch 5/80
8/8 [==============================] - 15s 2s/step - loss: 1.3119 - acc: 0.4141 - val_loss: 1.2184 - val_acc: 0.6071
Epoch 6/80
8/8 [==============================] - 15s 2s/step - loss: 1.2980 - acc: 0.4223 - val_loss: 1.1293 - val_acc: 0.5714
Epoch 7/80
8/8 [==============================] - 15s 2s/step - loss: 1.2850 - acc: 0.4143 - val_loss: 1.2361 - val_acc: 0.4286
Epoch 8/80
8/8 [==============================] - 16s 2s/step - loss: 1.2615 - acc: 0.4143 - val_loss: 1

8/8 [==============================] - 15s 2s/step - loss: 0.8066 - acc: 0.6932 - val_loss: 1.1352 - val_acc: 0.5714
Epoch 65/80
8/8 [==============================] - 15s 2s/step - loss: 0.8191 - acc: 0.6574 - val_loss: 1.0252 - val_acc: 0.6786
Epoch 66/80
8/8 [==============================] - 16s 2s/step - loss: 0.7666 - acc: 0.7171 - val_loss: 0.9197 - val_acc: 0.6071
Epoch 67/80
8/8 [==============================] - 15s 2s/step - loss: 0.7906 - acc: 0.7052 - val_loss: 0.8714 - val_acc: 0.6071
Epoch 68/80
8/8 [==============================] - 16s 2s/step - loss: 0.8078 - acc: 0.6614 - val_loss: 0.9422 - val_acc: 0.6071
Epoch 69/80
8/8 [==============================] - 16s 2s/step - loss: 0.7692 - acc: 0.6758 - val_loss: 0.9170 - val_acc: 0.4286
Epoch 70/80
8/8 [==============================] - 15s 2s/step - loss: 0.7879 - acc: 0.6375 - val_loss: 1.4276 - val_acc: 0.4286
Epoch 71/80
8/8 [==============================] - 15s 2s/step - loss: 0.8906 - acc: 0.6574 - val_loss: 1.040

In [9]:
model.evaluate(validation_generator,
steps=STEP_SIZE_TEST)

1/1 [==============================] - 0s 131ms/step - loss: 1.1444 - acc: 0.2500


[1.1444406509399414, 0.25]

In [10]:
# auto encoder 

In [11]:
type1_data.head()

,Name,Type1,imagepath
0,abomasnow,Grass,images/abomasnow.png
3,accelgor,Bug,images/accelgor.png
7,aipom,Normal,images/aipom.png
9,alomomola,Water,images/alomomola.png
12,ambipom,Normal,images/ambipom.png


In [12]:
dataset_type1 = pd.get_dummies(type1_data, columns=['Type1'], prefix='', prefix_sep='')
dataset_type1.head()

,Name,imagepath,Bug,Grass,Normal,Water
0,abomasnow,images/abomasnow.png,0,1,0,0
3,accelgor,images/accelgor.png,1,0,0,0
7,aipom,images/aipom.png,0,0,1,0
9,alomomola,images/alomomola.png,0,0,0,1
12,ambipom,images/ambipom.png,0,0,1,0


In [13]:
train_dataset = dataset_type1.sample(frac=0.8, random_state=0)
test_dataset = dataset_type1.drop(train_dataset.index)
train_dataset.head()

,Name,imagepath,Bug,Grass,Normal,Water
334,kangaskhan,images/kangaskhan.png,0,0,1,0
351,kricketune,images/kricketune.png,1,0,0,0
623,silvally,images/silvally.jpg,0,0,1,0
118,combee,images/combee.png,1,0,0,0
676,suicune,images/suicune.png,0,0,0,1
